### AWS Account Info

In [ ]:
import sagemaker

sess = sagemaker.Session()
role = 'arn:aws:iam::062336837736:role/Developer'
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

print(region)
print(role)
print(account)
print(sess.boto_session.profile_name)

### Build & Push Docker Image

#### Variables for Docker Image

In [ ]:
image = 'cog_verse'
bucket_name   = sess.default_bucket()
base_job_name = 'cog-verse-training'
%env image {image}
%env account {account}
%env region {region}
%env bucket_name {bucket_name}
%env base_job_name = {base_job_name}

#### Build Image

In [ ]:
# Enable/disable the docker build
is_build = "false"

In [ ]:
%%sh -s "$image" "$is_build"
bash ./build_and_push.sh $1 $2

#### Push Image to ECR

In [ ]:
!docker push $account.dkr.ecr.$region.amazonaws.com/${image}:latest

### Pack Source Code

In [ ]:
from cloud.sagemaker_utils import package_source_code, upload_to_s3, delete_archive
project_dir = "."
source_dir_names = [
    "actors",
    "cogment_verse",
    "config",
    "environments",
    "runs",
    "tests",
    "main.py",
    "simple_mlflow.py",
]
ignore_folders = ["node_modules"]
archive_name = "source_code.tar.gz"

# Pack all source code to run cogment verse
package_source_code(project_dir=project_dir, source_dir_names=source_dir_names, ignore_folders=ignore_folders, archive_name=archive_name)

# Upload to S3
s3_key = f"{image}/input/data/{archive_name}"
upload_to_s3(local_path=f"./{archive_name}", bucket=bucket_name, s3_key=s3_key)

# Delete packed source code after uploading to S3
delete_archive(archive_path=f"{project_dir}/{archive_name}")


### Training

#### User Inputs

In [ ]:
hyperparameters = {'main_args': "+experiment=ppo_atari_pz/pong_pz", 's3_bucket': bucket_name, "repo": image}

#### Local Test

In [ ]:
# Training setup
output_path = f"s3://{bucket_name}/{image}/output"
input_path = f"s3://{bucket_name}/{image}/input/data"
image_name = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:latest"

estimator = sagemaker.estimator.Estimator(image_uri=image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       instance_count=1, 
                       output_path=output_path,
                       instance_type='local',
                       hyperparameters=hyperparameters)
estimator.fit(inputs={"training": input_path})

# Verification
print(f"input_path: {input_path}")
print(f"output_path: {output_path}")
print(f"image_name: {image_name}")

#### AWS Run

In [ ]:
# Training setup
output_path = f"s3://{bucket_name}/{image}/output"
input_path = f"s3://{bucket_name}/{image}/input/data"
image_name = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:latest"
tag_name = [{'Key': 'cog-verse', 'Value': 'cog-verse-training'}]
base_job_name = 'cog-verse-training'

estimator = sagemaker.estimator.Estimator(image_uri=image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       instance_count=1, 
                       instance_type='ml.m5.xlarge',
                       tags=tag_name,
                       output_path=output_path,
                       sagemaker_session=sess,
                       hyperparameters=hyperparameters)
estimator.fit(inputs={"training": input_path})


# # Verification
# print(f"output_path: {output_path}")
# print(f"image_name: {image_name}")
